In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.ensemble import BaggingClassifier

In [25]:
train = pd.read_csv('./data/train.csv', index_col=0)
test = pd.read_csv('./data/test_x.csv', index_col=0)
submission = pd.read_csv('./data/sample_submission.csv', index_col=0)  

In [26]:
train_x = train.drop('voted', axis = 1)
train_y = train['voted']

In [54]:
X_train, X_vali, Y_train, Y_vali = train_test_split(train_x, train_y, test_size=0.25)

In [60]:
train_x_dummies = pd.get_dummies(X_train)
vali_x_dummies = pd.get_dummies(X_vali)

all_train_x_dummies = pd.get_dummies(train_x)
test_x_dummies = pd.get_dummies(test)

In [61]:
train_x_dummies.shape, vali_x_dummies.shape, all_train_x_dummies.shape, test_x_dummies.shape

((34149, 100), (11383, 100), (45532, 100), (11383, 100))

- train_val

In [64]:
lgbm_model = lgbm.LGBMClassifier(n_estimators=500)
lgbm_model.fit(train_x_dummies, Y_train)

LGBMClassifier(n_estimators=500)

In [65]:
roc_auc_score(Y_vali, list(pd.DataFrame(lgbm_model.predict_proba(vali_x_dummies)).apply(lambda x: x[1], axis=1)))

0.7512144128961044

- train_test

In [72]:
lgbm_model2 = lgbm.LGBMClassifier(n_estimators=500)
bagging_lgbm_model = BaggingClassifier(base_estimator=lgbm_model2, n_estimators=20).fit(all_train_x_dummies, train_y)

In [73]:
submission['voted']= list(pd.DataFrame(bagging_lgbm_model.predict_proba(test_x_dummies)).apply(lambda x: x[1], axis=1))

In [74]:
submission.to_csv('./data/sample_submission.csv')